In [1]:
from flask import Flask,render_template,request
import requests
import pickle
import Data_Preprocessing as dp
from tensorflow.keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from transformers import pipeline

In [2]:
svm_model = pickle.load(open('linearsvcmodel.pkl', 'rb'))
lstm_model = load_model('lstm_model.h5')
tok = pickle.load(open('tokenizer.pkl', 'rb'))
bert_model = pipeline("text-classification", model="output_dir")

In [3]:
mapping={'AE': 16,'BH': 17,'DZ': 10,'EG': 11,'IQ': 0,'JO': 6,'KW': 13,'LB': 12,'LY': 1,'MA': 7,'OM': 14,'PL': 3,
         'QA': 2,'SA': 8,'SD': 15,'SY': 4,'TN': 5,'YE': 9}

In [4]:
app = Flask(__name__)
@app.route('/', methods=['GET', 'POST'])
def home():
    return render_template('prediction.html')
@app.route('/prediction', methods=['GET','POST'])
def predict():
    text=request.form['text']
    text=dp.preprocesss(text)

    svm_pred = svm_model.predict([text])

    encoded = tok.texts_to_sequences([text])
    padded = pad_sequences(encoded,maxlen=280, padding='post')
    lstm_pred = np.argmax(lstm_model.predict(padded))
    lstm_pred = [k for k, v in mapping.items() if v == lstm_pred]
    bert_pred = bert_model(text)
    return render_template('prediction.html', prediction_string = 'Predictions :',
                           svm = 'SVM prediction : {}'.format(svm_pred[0]),
                           lstm = 'LSTM prediction : {}'.format(lstm_pred[0]),
                           arabert = 'Arabert prediction : {}'.format(bert_pred[0]['label']))
    
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2022 20:59:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 20:59:22] "GET /static/pred_style.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Mar/2022 20:59:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2022 21:00:31] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 21:00:31] "GET /static/pred_style.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Mar/2022 21:01:09] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 21:01:09] "GET /static/pred_style.css HTTP/1.1" 304 -
